In [2]:
library(RODBC)

In [3]:
# Establish database connection

dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            
dsn_hostname <- "b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" 
dsn_port <- "31249"   
dsn_protocol <- "TCPIP"            
dsn_uid <- "zmf98041"       
dsn_pwd <- "ysoZ11Qn2kaa9kRS"      
dsn_security <- "ssl"

conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")

conn <- odbcDriverConnect(conn_path)
conn

RODBC Connection 1
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=zmf98041
  PWD=******
  DATABASE=bludb
  HOSTNAME=b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud
  PORT=31249
  PROTOCOL=TCPIP
  SECURITY=SSL

In [4]:
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0800"

Driver_ODBC_Ver 
        "03.51"

In [5]:
#1 Determine how many records are in the seoul_bike_sharing dataset.
sqlQuery(conn, paste("SELECT COUNT(*) FROM SEOUL_BIKE_SHARING;"))

,1
,<int>
1,8465


In [6]:
#2 Determine how many hours had non-zero rented bike count.
sqlQuery(conn, paste("SELECT COUNT(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT != 0;"))

,1
,<int>
1,8465


In [7]:
#3 Query the weather forecast for Seoul over the next 3 hours.
sqlQuery(conn, paste("SELECT * FROM CITIES_WEATHER_FORECAST WHERE CITY = 'Seoul' LIMIT 1;"))

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


In [8]:
#4 Find which seasons are included in the seoul bike sharing dataset.
sqlQuery(conn, paste("SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING"))

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


In [9]:
#5 Find the first and last dates in the Seoul Bike Sharing dataset. NOT DONE YET!!
sqlQuery(conn, paste("SELECT * FROM SEOUL_BIKE_SHARING LIMIT 1"))
# sqlQuery(conn, paste("SELECT LAST() FROM SEOUL_BIKE_SHARING"))


,DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0,0,0,Winter,No Holiday,Yes


[1] "42884 -440 [IBM][CLI Driver][DB2/LINUXX8664] SQL0440N  No authorized routine named \"LAST\" of type \"FUNCTION\" having compatible arguments was found.  SQLSTATE=42884\n"
[2] "[RODBC] ERROR: Could not SQLExecDirect 'SELECT LAST() FROM SEOUL_BIKE_SHARING'"

In [10]:
#6 Determine which date and hour had the most bike rentals.
sqlQuery(conn, paste("SELECT DATE, HOUR, RENTED_BIKE_COUNT FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT = ( SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING );"))

,DATE,HOUR,RENTED_BIKE_COUNT
,<fct>,<int>,<int>
1,19/06/2018,18,3556


In [11]:
#7 Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.
# sqlQuery(conn, paste("SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS RENTED_BIKE_COUNT_AVERAGE, AVG(TEMPERATURE) AS AVERAGE_TEMPERATURE FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"))  NOT DONE YET!
sqlQuery(conn, paste("SELECT SEASONS, HOUR, AVG(RENTED_BIKE_COUNT) AS RENTED_BIKE_COUNT_AVERAGE, AVG(TEMPERATURE) AS AVERAGE_TEMPERATURE FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"))

[1] "42803 -119 [IBM][CLI Driver][DB2/LINUXX8664] SQL0119N  An expression starting with \"HOUR\" specified in a SELECT clause, HAVING clause, or ORDER BY clause is not specified in the GROUP BY clause or it is in a SELECT clause, HAVING clause, or ORDER BY clause with a column function and no GROUP BY clause is specified.  SQLSTATE=42803\n"
[2] "[RODBC] ERROR: Could not SQLExecDirect 'SELECT SEASONS, HOUR, AVG(RENTED_BIKE_COUNT) AS RENTED_BIKE_COUNT_AVERAGE, AVG(TEMPERATURE) AS AVERAGE_TEMPERATURE FROM SEOUL_BIKE_SHARING GROUP BY SEASONS'"

In [13]:
#8 Find the average hourly bike count during each season. Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.
sqlQuery(conn, paste("SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS AVERAGE_HOURLY_BIKE_COUNT, MIN(RENTED_BIKE_COUNT) AS MINIMUN_VALUE, MAX(RENTED_BIKE_COUNT) AS MAXIMUM_VALUE, STDDEV(RENTED_BIKE_COUNT) AS STANDARD_DEVIATION FROM SEOUL_BIKE_SHARING GROUP BY SEASONS"))

,SEASONS,AVERAGE_HOURLY_BIKE_COUNT,MINIMUN_VALUE,MAXIMUM_VALUE,STANDARD_DEVIATION
,<fct>,<int>,<int>,<int>,<dbl>
1,Autumn,924,2,3298,617.3885
2,Spring,746,2,3251,618.5247
3,Summer,1034,9,3556,690.0884
4,Winter,225,3,937,150.3374


In [14]:
#9 Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season? # Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.
sqlQuery(conn, paste("SELECT SEASONS, AVG(RENTED_BIKE_COUNT) AS AVERAGE_BIKE_COUNT, AVG(TEMPERATURE) AS AVERAGE_TEMPERATURE, AVG(HUMIDITY) AS AVERAGE_HUMIDITY, AVG(WIND_SPEED) AS AVERAGE_WINDSPEED, AVG(VISIBILITY) AS AVERAGE_VISIBILITY, AVG(DEW_POINT_TEMPERATURE) AS AVAERAGE_DEWPOINT_TEMPERATURE, AVG(SOLAR_RADIATION) AS AVERAGE_SOLAR_RADIATION, AVG(RAINFALL) AS AVERAGE_RAINFALL, AVG(SNOWFALL) AS AVERAGE_SNOWFALL FROM SEOUL_BIKE_SHARING GROUP BY SEASONS ORDER BY AVERAGE_BIKE_COUNT"))


,SEASONS,AVERAGE_BIKE_COUNT,AVERAGE_TEMPERATURE,AVERAGE_HUMIDITY,AVERAGE_WINDSPEED,AVERAGE_VISIBILITY,AVAERAGE_DEWPOINT_TEMPERATURE,AVERAGE_SOLAR_RADIATION,AVERAGE_RAINFALL,AVERAGE_SNOWFALL
,<fct>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Winter,225,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000
2,Spring,746,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000
3,Autumn,924,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026
4,Summer,1034,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000


In [15]:
#10 Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view. Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.
sqlQuery(conn, paste("SELECT WORLD_CITIES.CITY, WORLD_CITIES.COUNTRY, LAT, LNG, POPULATION, BICYCLES FROM WORLD_CITIES, BIKE_SHARING_SYSTEMS WHERE WORLD_CITIES.CITY = BIKE_SHARING_SYSTEMS.CITY AND WORLD_CITIES.CITY = 'Seoul'"))


,CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


In [16]:
#11 Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.
sqlQuery(conn, paste("SELECT WORLD_CITIES.CITY, WORLD_CITIES.COUNTRY,LAT, LNG, POPULATION, BICYCLES FROM WORLD_CITIES, BIKE_SHARING_SYSTEMS WHERE WORLD_CITIES.CITY = BIKE_SHARING_SYSTEMS.CITY AND WORLD_CITIES.COUNTRY = BIKE_SHARING_SYSTEMS.COUNTRY AND BICYCLES BETWEEN 15000 AND 20000"))





CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
